In [73]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam
import openpyxl
import prettytable as pt

In [74]:
#ENTER DESIRED GAMEWEEK FOR PREDICTIONS
GW=1

In [75]:
# importing the tools required for the Poisson regression model

epl = pd.read_csv(r"C:\Users\blake\OneDrive\Documents\Pl_2223 xG.csv",encoding='latin1')
ep = epl[['Home','Away','xGh','xGa']]
epl = epl.rename(columns={'xGh': 'HomeGoals', 'xGa': 'AwayGoals'})

In [76]:
# Replace "Leicester City" with "Burnley" in the Home variable
epl['Home'].replace('Leicester City', 'Burnley', inplace=True)

    # Replace "Leicester City" with "Burnley" in the Away variable
epl['Away'].replace('Leicester City', 'Burnley', inplace=True)

# Replace "Leeds United" with "Sheffield United" in the Home variable
epl['Home'].replace('Leeds United', 'Sheffield United', inplace=True)

    # Replace "Leeds United" with "Burnley" in the Away variable
epl['Away'].replace('Leeds United', 'Sheffield United', inplace=True)

# Replace "Southampton" with "Luton Town" in the Home variable
epl['Home'].replace('Southampton', 'Luton Town', inplace=True)

    # Replace "Southampton" with "Luton Town" in the Away variable
epl['Away'].replace('Southampton', 'Luton Town', inplace=True)


In [77]:
# Create a Poisson regression model
home_model = smf.glm(formula="HomeGoals ~ + Home + Away", data=epl, family=sm.families.Poisson()).fit()

# Create a Poisson regression model for the away team
away_model = smf.glm(formula="AwayGoals ~ + Home + Away", data=epl, family=sm.families.Poisson()).fit()


In [78]:
# Read the raw data into a DataFrame
fixtures = pd.read_excel(r"C:\Users\blake\Premier-League-2023_2024-Schedule 2.xlsx")

In [79]:

fixtures['Home'].replace('Man City', 'Manchester City', inplace=True)
fixtures['Away'].replace('Man City', 'Manchester City', inplace=True)

fixtures['Home'].replace('Man UTD', 'Manchester Utd', inplace=True)
fixtures['Away'].replace('Man UTD', 'Manchester Utd', inplace=True)

fixtures['Home'].replace('Newcastle', 'Newcastle Utd', inplace=True)
fixtures['Away'].replace('Newcastle', 'Newcastle Utd', inplace=True)

fixtures['Home'].replace('Nottingham', "Nott'ham Forest", inplace=True)
fixtures['Away'].replace('Nottingham', "Nott'ham Forest", inplace=True)

fixtures['Home'].replace('Luton', "Luton Town", inplace=True)
fixtures['Away'].replace('Luton', "Luton Town", inplace=True)

fixtures['Home'].replace('Sheffield UTD', "Sheffield United", inplace=True)
fixtures['Away'].replace('Sheffield UTD', "Sheffield United", inplace=True)

In [80]:
# Select the rows where the Matchday is x
df = fixtures.query("Matchday == @GW")

In [81]:
print(df)

  Matchday       Date              Home             Away  Result
0        1 2023-08-11           Burnley  Manchester City     NaN
1        1 2023-08-12           Arsenal  Nott'ham Forest     NaN
2        1 2023-08-12  Sheffield United   Crystal Palace     NaN
3        1 2023-08-12       Bournemouth         West Ham     NaN
4        1 2023-08-12          Brighton       Luton Town     NaN
5        1 2023-08-12           Everton           Fulham     NaN
6        1 2023-08-12     Newcastle Utd      Aston Villa     NaN
7        1 2023-08-13         Brentford        Tottenham     NaN
8        1 2023-08-13           Chelsea        Liverpool     NaN
9        1 2023-08-14    Manchester Utd           Wolves     NaN


In [82]:
import scipy.stats as stats

# Predict the score of each game in the DataFrame 
for index, row in df.iterrows():

  # Home team prediction
  home_pred = home_model.predict(epl) 

  # Away team prediction
  away_pred = away_model.predict(epl)
               
  # Save predictions
  home_score_rate = home_pred
  away_score_rate = away_pred

# Add the predictions to the DataFrame
df['home_xG'] = home_pred.round(2)
df['away_xG'] = away_pred.round(2)



C:\Users\blake\AppData\Local\Temp\ipykernel_44028\973020967.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['home_xG'] = home_pred.round(2)
C:\Users\blake\AppData\Local\Temp\ipykernel_44028\973020967.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['away_xG'] = away_pred.round(2)


In [83]:
# Create a new column called home_score
df['home_goals'] = [np.random.poisson(home_pred[i]) for i in range(len(df))]

# Create a new column called away_score
df['away_goals'] = [np.random.poisson(away_pred[i]) for i in range(len(df))]

C:\Users\blake\AppData\Local\Temp\ipykernel_44028\2088413016.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['home_goals'] = [np.random.poisson(home_pred[i]) for i in range(len(df))]
C:\Users\blake\AppData\Local\Temp\ipykernel_44028\2088413016.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['away_goals'] = [np.random.poisson(away_pred[i]) for i in range(len(df))]


In [84]:
# Create a table of the results
table = df[['Matchday', 'Home','home_goals','away_goals', 'Away', 'home_xG', 'away_xG']]
print(table.to_string())

  Matchday              Home  home_goals  away_goals             Away  home_xG  away_xG
0        1           Burnley           1           1  Manchester City     0.82     1.46
1        1           Arsenal           2           1  Nott'ham Forest     1.35     2.06
2        1  Sheffield United           3           1   Crystal Palace     1.68     0.79
3        1       Bournemouth           5           0         West Ham     2.52     0.57
4        1          Brighton           4           1       Luton Town     1.49     1.13
5        1           Everton           0           1           Fulham     1.11     1.28
6        1     Newcastle Utd           0           3      Aston Villa     1.20     1.29
7        1         Brentford           3           0        Tottenham     1.08     1.39
8        1           Chelsea           1           0        Liverpool     1.90     1.40
9        1    Manchester Utd           0           1           Wolves     0.95     1.89


In [85]:
for index, row in df.iterrows():

  # Calculate the probability of a home win
  home_win_prob = stats.poisson.pmf(row['home_goals'], home_score_rate[index])

  # Calculate the probability of a draw
  draw_prob = stats.poisson.pmf(row['home_goals'], home_score_rate[index]) * stats.poisson.pmf(row['away_goals'], away_score_rate[index])

  # Calculate the probability of an away win
  away_win_prob = 1 - home_win_prob - draw_prob

  # Set the value of the cell at index and column 'home_win_prob'
  df.at[index, 'home_win_prob'] = (home_win_prob*100).round(2)

  # Set the value of the cell at index and column 'draw_prob'
  df.at[index, 'draw_prob'] = (draw_prob*100).round(2)

  # Set the value of the cell at index and column 'away_win_prob'
  df.at[index, 'away_win_prob'] = (away_win_prob*100).round(2)
    
print(df)

  Matchday       Date              Home             Away  Result  home_xG  \
0        1 2023-08-11           Burnley  Manchester City     NaN     0.82   
1        1 2023-08-12           Arsenal  Nott'ham Forest     NaN     1.35   
2        1 2023-08-12  Sheffield United   Crystal Palace     NaN     1.68   
3        1 2023-08-12       Bournemouth         West Ham     NaN     2.52   
4        1 2023-08-12          Brighton       Luton Town     NaN     1.49   
5        1 2023-08-12           Everton           Fulham     NaN     1.11   
6        1 2023-08-12     Newcastle Utd      Aston Villa     NaN     1.20   
7        1 2023-08-13         Brentford        Tottenham     NaN     1.08   
8        1 2023-08-13           Chelsea        Liverpool     NaN     1.90   
9        1 2023-08-14    Manchester Utd           Wolves     NaN     0.95   

   away_xG  home_goals  away_goals  home_win_prob  draw_prob  away_win_prob  
0     1.46           1           1          36.13      12.24          51.6

C:\Users\blake\AppData\Local\Temp\ipykernel_44028\1161913554.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, 'home_win_prob'] = (home_win_prob*100).round(2)
C:\Users\blake\AppData\Local\Temp\ipykernel_44028\1161913554.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, 'draw_prob'] = (draw_prob*100).round(2)
C:\Users\blake\AppData\Local\Temp\ipykernel_44028\1161913554.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [86]:
# Create a PrettyTable object
table = pt.PrettyTable()

# Add the headers
table.field_names = ['ID','GW', 'Home Team', 'Home Goals', 'Away Goals', 'Away Team', 'Home Win %', 'Draw %', 'Away Win %']

for i in range(len(df)):
    row = [i+1, df['Matchday'].iloc[i], df['Home'].iloc[i], df['home_goals'].iloc[i], df['away_goals'].iloc[i], df['Away'].iloc[i], df['home_win_prob'].iloc[i], df['draw_prob'].iloc[i], df['away_win_prob'].iloc[i]] 
    
    table.add_row(row)

# Print the table
print(table.get_string())

+----+----+------------------+------------+------------+-----------------+------------+--------+------------+
| ID | GW |    Home Team     | Home Goals | Away Goals |    Away Team    | Home Win % | Draw % | Away Win % |
+----+----+------------------+------------+------------+-----------------+------------+--------+------------+
| 1  | 1  |     Burnley      |     1      |     1      | Manchester City |   36.13    | 12.24  |   51.63    |
| 2  | 1  |     Arsenal      |     2      |     1      | Nott'ham Forest |   23.62    |  6.2   |   70.18    |
| 3  | 1  | Sheffield United |     3      |     1      |  Crystal Palace |   14.77    |  5.3   |   79.93    |
| 4  | 1  |   Bournemouth    |     5      |     0      |     West Ham    |    6.82    |  3.85  |   89.33    |
| 5  | 1  |     Brighton     |     4      |     1      |    Luton Town   |    4.6     |  1.68  |   93.71    |
| 6  | 1  |     Everton      |     0      |     1      |      Fulham     |   32.84    | 11.68  |   55.48    |
| 7  | 1  